In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
from datasets import load_dataset

dataset = load_dataset('knowledgator/events_classification_biotech') 
    
classes = [class_ for class_ in dataset['train'].features['label 1'].names if class_]
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset['train'].to_pandas()

,title,content,target organization,all_labels,all_labels_concat,label 1,label 2,label 3,label 4,label 5
0,Sarah Polley's Book Recommendations,Drive Your Plow Over the Bones of The Dead\nby...,Franny's Farmacy,[other],other,23,NaN,NaN,NaN,NaN
1,Denel staff get millions from attached bank ac...,"In the recently tabled National Budget, Denel ...",Heat Relief,[other],other,23,NaN,NaN,NaN,NaN
2,How to master productive pausing and get more ...,Shares\nTake a break its good for you (Picture...,Reframe,[other],other,23,NaN,NaN,NaN,NaN
3,Early Bird & Eight Certifications! RESO Weekly...,RESO is currently hiring for two positions:\nP...,CARE SOUTH,[other],other,23,NaN,NaN,NaN,NaN
4,The Lifetime Discount Club,Charter Buyer Club\nWhat is the Charter Buyer ...,Big Sky Botanicals,[other],other,23,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2754,Birmingham Mail backs 2025 Invictus Games bid,0\nA regional daily wants to bring an internat...,Invictus Games,"[support & philanthropy, company description]","support & philanthropy, company description",26,21.0,NaN,NaN,NaN
2755,R1 RCM Inc. (RCM) Reveals an Earnings Mystery,Share on whatsapp\nR1 RCM Inc. (NASDAQ:RCM)\nw...,ABCS RCM,"[investment in public company, company descrip...","investment in public company, company description",22,21.0,NaN,NaN,NaN
2756,Verboso Launches Full-Stack Online Speech Ther...,Verboso Launches Full-Stack Online Speech Ther...,Verboso,[product launching & presentation],product launching & presentation,14,NaN,NaN,NaN,NaN
2757,"Barnet, Enfield and Haringey Mental Health Tru...","Barnet, Enfield and Haringey Mental Health Tru...",Barnet Enfield and Haringey Mental Health Trust,[executive statement],executive statement,1,NaN,NaN,NaN,NaN


In [4]:
from transformers import AutoTokenizer

model_path = 'microsoft/deberta-v3-small'

tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

In [5]:
def preprocess_function(example):
    
    text = f"{example['title']}.\n{example['content']}"

    
    all_labels = example['all_labels']
    labels = [0. for i in range(len(classes))]

    for label in all_labels:
        label_id = class2id[label]
        labels[label_id] = 1.

    example = tokenizer(text, truncation=True)
    example['labels'] = labels
    return example

tokenized_dataset = dataset.map(preprocess_function)


Map:   0%|          | 0/2759 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/381 [00:00<?, ? examples/s]

In [6]:
dataset['train']

Dataset({
    features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5'],
    num_rows: 2759
})

In [7]:
tokenized_dataset['train']

Dataset({
    features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 2759
})

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int).reshape(-1)
    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))
    references=labels.astype(int).reshape(-1)


In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(

   model_path, num_labels=len(classes),
           id2label=id2class, label2id=class2id,
                       problem_type = "multi_label_classification")

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import os
os.environ["WANDB_PROJECT"]="Dilbazlar"

In [12]:
wandb_api_key = "04a083b14d60688b24482e00727ebcc57448ef88"

In [13]:
training_args = TrainingArguments(
   output_dir="my_awesome_model",
   learning_rate=2e-5,
   per_device_train_batch_size=3,
   per_device_eval_batch_size=3,
   num_train_epochs=2,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   report_to="wandb", # Wandb = https://docs.wandb.ai/guides/integrations/huggingface
   run_name="Multi-label-model-baseline"
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\halilibrahim.hatun\_netrc


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.231700,0.146006,0.949588,0.367764,0.720000,0.246951
2,0.148800,0.134350,0.955833,0.512000,0.744186,0.390244


NameError: name 'wandb' is not defined